In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_vendor_by_dc_sku 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH
sellin_price_cur AS (
    SELECT
        a.* EXCEPT(vendor_subrange),
	    IFNULL(IF(a.vendor_subrange = '' OR a.vendor_subrange IS NULL, c.subrange_id, a.vendor_subrange), '') vendor_subrange
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_sellin_price_cur a
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_vendor_master b ON a.vendor = b.vendor_id
        INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_pir_current c ON LTRIM('0', a.vendor) = c.vendor_id AND a.product_id = c.product_id
        INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product_unit d
            ON a.product_id = d.product_id AND a.purchasing_unit = d.unit
    WHERE IFNULL(b.block_company_code, 'Y') <> 'X' 
        AND IFNULL(b.block_purchasing_org, 'Y') <> 'X'
        AND a.cond_table <> '913'
),
sellin_price AS (
    SELECT 
        a.* EXCEPT(price_from, price_to), 
        price_from valid_from, 
        price_to valid_to,
        CASE 
            WHEN TRIM(IFNULL(a.store_id, '') || IFNULL(a.supply_region, '') || IFNULL(a.site_region, '')) <> '' 
            THEN TRIM(IFNULL(a.store_id, '') || IFNULL(a.supply_region, '') || IFNULL(a.site_region, '')) 
        ELSE '1401' 
        END varkey,
        pri.pri priority,
        CASE
            WHEN a.store_id IS NOT NULL THEN 1
            WHEN a.site_region IS NOT NULL THEN 2
            WHEN a.supply_region IS NOT NULL THEN 3
            ELSE 4
        END priority_level
    FROM sellin_price_cur a 
        LEFT JOIN {catalog_name}.udp_wcm_silver_dim.sellin_pri pri ON pri.table = CAST(a.cond_table AS BIGINT)
    WHERE IFNULL(a.info_type, '') <> '' 
        AND a.purchasing_org = '1401' 
        AND IFNULL(a.purchasing_group, '101') NOT IN ('601','602') 
),
sellin_price_bysite_prev AS (
	SELECT 
		sp.*,
		IFNULL(IFNULL(IFNULL(sp.store_id, st1.store_id), st2.store_id), st3.store_id) site
	FROM sellin_price sp 
		LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store st1 ON sp.varkey = '1401'
		LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store st2 ON sp.varkey = st2.supply_region
		LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store st3 ON sp.varkey = st3.region
),
sup_region AS (
    SELECT DISTINCT vendor
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_sup_region
),
sellin_price_bysite AS (
    SELECT DISTINCT *
    FROM (
        SELECT a.*
        FROM sellin_price_bysite_prev a
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_sup_region b
            ON LTRIM('0', a.vendor) = LTRIM('0', b.vendor)
        WHERE b.vendor IS NULL
        UNION ALL
        
        SELECT DISTINCT a.*
        FROM sellin_price_bysite_prev a
        INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_sup_region b
            ON LTRIM('0', a.vendor) = LTRIM('0', b.vendor)
        INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store c
            ON a.site = c.store_id AND c.supply_region = b.supply_region
        WHERE a.priority_level IN (1, 2)
        UNION ALL
       
        SELECT a.*
        FROM sellin_price_bysite_prev a
        INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_sup_region b
            ON LTRIM('0', a.vendor) = LTRIM('0', b.vendor) AND a.supply_region = b.supply_region
        WHERE a.priority_level = 3
        UNION ALL
        SELECT a.*
        FROM sellin_price_bysite_prev a
        INNER JOIN sup_region b
            ON LTRIM('0', a.vendor) = LTRIM('0', b.vendor)
        WHERE a.priority_level = 4
    )
),
po AS (
    SELECT DISTINCT 
        f.store_id, 
        f.product_id, 
        f.vendor,
        f.po_unit,
        FIRST_VALUE(f.po_id) OVER (PARTITION BY f.store_id, f.product_id, f.vendor, f.po_unit ORDER BY f.gr_date DESC) po_id,
        FIRST_VALUE(f.gr_date) OVER (PARTITION BY f.store_id, f.product_id, f.vendor, f.po_unit ORDER BY f.gr_date DESC) max_grdate
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_po_item f
    WHERE f.vendor IS NOT NULL 
        AND UPPER(f.po_type) IN ('COLLECTIVE PO', 'STANDARD PO') 
        AND f.gr_qty > 0
),
po_by_province AS (
    SELECT 
        DISTINCT
        s.region,
        f.product_id,
        f.vendor,
        f.po_unit,
        FIRST_VALUE(f.po_id) OVER (PARTITION BY s.region, f.product_id, f.vendor, f.po_unit ORDER BY f.max_grdate DESC) po_id,
        FIRST_VALUE(f.max_grdate) OVER (PARTITION BY s.region, f.product_id, f.vendor, f.po_unit ORDER BY f.max_grdate DESC) max_grdate
    FROM po f
    INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store s
        ON f.store_id = s.store_id
    WHERE s.business_unit = '1100'
),
po_sub_mnf AS (
    SELECT DISTINCT 
        f.store_id, 
        p.sub_manufacturer_id, 
        f.vendor,
        FIRST_VALUE(f.po_id) OVER (PARTITION BY f.store_id, p.sub_manufacturer_id, f.vendor ORDER BY f.gr_date DESC) po_id_sub_mnf,
        FIRST_VALUE(f.gr_date) OVER (PARTITION BY f.store_id, p.sub_manufacturer_id, f.vendor ORDER BY f.gr_date DESC) max_grdate_sub_mnf
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_po_item f
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product p
        ON f.product_id = p.product_id
    WHERE f.vendor IS NOT NULL 
        AND UPPER(f.po_type) IN ('COLLECTIVE PO', 'STANDARD PO') 
        AND f.gr_qty > 0
),
po_by_province_sub_mnf AS (
    SELECT 
        DISTINCT
        s.region,
        f.sub_manufacturer_id,
        f.vendor,
        FIRST_VALUE(f.po_id_sub_mnf) OVER (PARTITION BY s.region, f.sub_manufacturer_id, f.vendor ORDER BY f.max_grdate_sub_mnf DESC) po_id_sub_mnf,
        FIRST_VALUE(f.max_grdate_sub_mnf) OVER (PARTITION BY s.region, f.sub_manufacturer_id, f.vendor ORDER BY f.max_grdate_sub_mnf DESC) max_grdate_sub_mnf
    FROM po_sub_mnf f
    INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store s
        ON f.store_id = s.store_id
    WHERE s.business_unit = '1100'
),
price AS (
    SELECT DISTINCT 
        a.site as dc_id,
        st.supply_region,
        st.region,
        a.product_id,
        a.purchasing_unit as uom, 
        a.varkey as price_level,
        a.priority_level,
        a.sell_price,
        a.valid_from, 
        a.valid_to,
        a.vendor, 
        a.vendor_subrange, 
        a.priority as cond_pri,
        IFNULL(p1.max_grdate, p2.max_grdate) as max_grdate,
        IFNULL(p3.max_grdate_sub_mnf, p4.max_grdate_sub_mnf) as max_grdate_sub_mnf,
        DENSE_RANK() OVER (PARTITION BY a.site, a.product_id, a.purchasing_unit ORDER BY a.priority, COALESCE(p1.max_grdate, p2.max_grdate, '2016-01-01') DESC, COALESCE(p3.max_grdate_sub_mnf, p4.max_grdate_sub_mnf, '2016-01-01') DESC, a.valid_from DESC) priority
    FROM sellin_price_bysite a
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store st ON st.store_id = a.site
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product pd ON pd.product_id = a.product_id
        LEFT JOIN po p1 ON p1.store_id = a.site AND p1.product_id = a.product_id AND p1.vendor = a.vendor AND p1.po_unit = a.purchasing_unit
        LEFT JOIN po_by_province p2 ON p2.region = st.region AND p2.product_id = a.product_id AND p2.vendor = a.vendor AND p2.po_unit = a.purchasing_unit
        LEFT JOIN po_sub_mnf p3 ON p3.store_id = a.site AND p3.sub_manufacturer_id = pd.sub_manufacturer_id AND p3.vendor = a.vendor
        LEFT JOIN po_by_province_sub_mnf p4 ON p4.region = st.region AND p4.sub_manufacturer_id = pd.sub_manufacturer_id AND p4.vendor = a.vendor
    WHERE 
        DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) BETWEEN a.valid_from AND a.valid_to
        AND st.store_status_id <> 'closed'
        AND st.business_unit = '1100'
),
main AS (
	SELECT *
	FROM price
	WHERE IFNULL(priority, 1) = 1
		AND uom NOT IN ('KGM', 'LTR', 'MTR')
)
SELECT *
FROM main;
""")